# Train

Training various models, following this [Kaggle guide](https://www.kaggle.com/code/merturper/breast-cancer-outliers-pca-nca/notebook#Train-Test-Split-&-StandardScaler). Pushing results to our "Wisconsin BCa" experiment in MLFlow.

Note that the `train` folder contains code to package and train the model on the cloud, while this is just playing around.

## Read Data 

Read both boxplot and lof preprocessed data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis, LocalOutlierFactor
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import mlflow
import mlflow.sklearn

In [2]:
# get a handle of the data asset and print the URI
data_boxplot = './data/cleaned-wisconsin-boxplot.parquet'
data_lof = './data/cleaned-wisconsin-lof.parquet'

df_boxplot = pd.read_parquet(data_boxplot)
df_lof = pd.read_parquet(data_boxplot)

# Setup MLFLow


In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

Found the config file in: ./config.json
Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [4]:
mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri


DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines here at https://aka.ms/azsdk/py

ClientAuthenticationError: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Azure CLI not found on path
	AzurePowerShellCredential: PowerShell is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines here

Request time out. Ingestion may be backed up. Retrying.


## Define

In [3]:
def split_and_scale(df):
    y = df["diagnosis_01"]
    X = df.drop(["diagnosis_01"],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test
    

In [10]:
def train_logistic(X_train, X_test, y_train, y_test):
    log_reg = LogisticRegression()
    log_reg.fit(X_train, y_train)
    # model predictions
    y_pred = log_reg.predict(X_test)
    mlflow.log_metrics({
        'trainScore': accuracy_score(y_train, log_reg.predict(X_train)),
        'testScore': accuracy_score(y_test, log_reg.predict(X_test))
    })
    mlflow.log_dict(np.array(confusion_matrix(y_test, y_pred)).tolist(), artifact_file='confusion_matrix.json')
    mlflow.log_dict(classification_report(y_test, y_pred), 'classification_report.json')

In [11]:
def train_knn(X_train, X_test, y_train, y_test):
    knn = KNeighborsClassifier(n_neighbors = 3)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    mlflow.log_metrics({
        'trainScore': accuracy_score(y_train, knn.predict(X_train)),
        'testScore': accuracy_score(y_test, knn.predict(X_test))
    })
    mlflow.log_dict(np.array(confusion_matrix(y_test, y_pred)).tolist(), artifact_file='confusion_matrix.json')
    mlflow.log_dict(classification_report(y_test, y_pred), 'classification_report.json')

In [12]:
def train_svc(X_train, X_test, y_train, y_test):
    svc = SVC()
    svc.fit(X_train, y_train)
    # model predictions 
    y_pred = svc.predict(X_test)
    mlflow.log_metrics({
        'trainScore': accuracy_score(y_train, svc.predict(X_train)),
        'testScore': accuracy_score(y_test, svc.predict(X_test))
    })
    mlflow.log_dict(np.array(confusion_matrix(y_test, y_pred)).tolist(), artifact_file='confusion_matrix.json')
    mlflow.log_dict(classification_report(y_test, y_pred), 'classification_report.json')

## Run

In [16]:
# set name for logging
mlflow.set_experiment("Wisconsin BCa Experiment local")
mlflow.autolog()

2023/05/12 20:37:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xfffeec0fdb80>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
2023/05/12 20:37:10 INFO mlflow.tracking.fluent: Autologging successfully ena

In [14]:
# random helper function
# https://stackoverflow.com/a/11146645
def cartesian_product(x, y):
    return [(x0, y0) for x0 in x for y0 in y]

In [19]:
with mlflow.start_run():
    dfs = [(df_boxplot, 'boxplot'), (df_lof, 'lof')]
    trains = [(train_knn, 'knn'), (train_logistic, 'logistic'), (train_svc, 'svc')]

    mlflow.log_param("parent", "yes")
    for (df, df_name), (train, train_name) in [(x0, y0) for x0 in dfs for y0 in trains]:
        with mlflow.start_run(run_name=f'{df_name}-outlier-with-{train_name}', nested=True):
            mlflow.set_tag('outlier_func', df_name)
            X_train, X_test, y_train, y_test = split_and_scale(df)
            train(X_train, X_test, y_train, y_test)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0xffff3df16c10>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/opt/conda/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
/opt/conda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior

# Viewing the models

See the models by running `mlflow ui`. I might connect this with azure...

In [ ]:
X_train, X_test, y_train, y_test = split_and_scale(df_lof)
print(len(X_test[0]))
list(df_lof.columns[:-1])

26


['texture_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'radius_se',
 'texture_se',
 'perimeter_se',
 'area_se',
 'smoothness_se',
 'compactness_se',
 'concavity_se',
 'concave points_se',
 'symmetry_se',
 'fractal_dimension_se',
 'texture_worst',
 'perimeter_worst',
 'area_worst',
 'smoothness_worst',
 'compactness_worst',
 'concavity_worst',
 'concave points_worst',
 'symmetry_worst',
 'fractal_dimension_worst']

## Summary

The Kaggle guy says that SVC gives the best results. I'm going to just use SVC and deploy it, comparing results is a bit weird on Azure somehow.

##